## Локальное развертывание llamav2 3B

In [2]:
%pip install transformers huggingface_hub --q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# model_name = "abdulrahman-nuzha/finetuned-llama-v2.0"
# model_name='RoversX/llama-2-7b-hf-small-shards-Samantha-V1-SFT'
# model_name='yeen214/llama2_7b_merge_orcafamily'
#...

In [1]:
from tqdm.notebook import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = 'openlm-research/open_llama_3b'
model = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'auto')
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map = 'auto')

config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
prompts_meanings = [
"Can you explain what is ology?",
"Can you explain what is playset?",
"Can you explain what is footwell?",
"Can you explain what is fathering?",
"Can you explain what is conjunctiva?",
"Can you explain what is seediness?",
"Can you explain what is followership?",
"Can you explain what is restenosis?",
"Can you explain what is lectionary?",
"Can you explain what is soundwave?",
"Can you explain what is applecart?",
"Can you explain what is bass-baritone?",
"Can you explain what is humidification?",
"Can you explain what is billon?",
"Can you explain what is mousehole?",
"Can you explain what is signaler?",
"Can you explain what is malamute?",
"Can you explain what is siltstone?",
"Can you explain what is adventuress?",
"Can you explain what is nerdiness?"
]

### baseline - greedy strategy

Для запросов о **значениях терминов:**

In [3]:
output_texts = []
for prompt in tqdm(prompts_meanings):
    input_text = 'Q: ' + prompt
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

    max_length = 300  # set the maximum length of the generated sequence

    output_ids = model.generate(
        input_ids,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,  # specify the end-of-sequence token ID
    )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    output_texts.append(output_text)

  0%|          | 0/20 [00:00<?, ?it/s]

**Примеры плохих ответов модели**, где она не может дать какой-либо логичный ответ (скорее всего эти галлюцинации нельзя починить, так как модель в целом не понимает вопроса)

Q: Can you explain what is playset? 

A: I'm trying to understand what is playset.
I've read the documentation and I'm still not sure what is playset.

Q: Can you explain what is conjunctiva?

A: I have a question about the conjunctiva.

**Примеры, которые можно попробовать починить**

Q: Can you explain what is ology?

A: Ology is a word that is used to describe a particular type of blog. It is a portmanteau of "blog" and "ology".

Q: Can you explain what is billon? 

A: The term billon is used to describe a coin that was struck in a small quantity. The term is used to describe coins struck in the 1st century BC and 1st century AD

Q: Can you explain what is malamute?

A: Malamute is a breed of dog that is a combination of Siberian Husky and Alaskan Malamute.

Запросы для **абстрактного мышления:**

In [6]:
prompts_logic = [
"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: If the squirrel likes the tiger and something does not see the squirrel then the squirrel likes the dog. The dog is big. The bald eagle likes the dog. The dog likes the squirrel. The tiger chases the dog. The dog does not chase the squirrel. The bald eagle likes the squirrel. The bald eagle does not chase the tiger. If something sees the squirrel then the squirrel does not like the bald eagle. If something likes the dog then it chases the squirrel. If something chases the squirrel then the squirrel likes the tiger. The dog likes the bald eagle. If something sees the bald eagle then the bald eagle is not blue. The dog does not chase the bald eagle. The tiger sees the squirrel. The squirrel does not like the bald eagle. The dog likes the tiger. The dog sees the bald eagle. If something chases the bald eagle then the bald eagle does not chase the dog. The squirrel chases the tiger. The dog does not see the tiger. The bald eagle is not blue. Query: The dog sees the bald eagle.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: Charlie is blue. All kind things are red. All blue, red things are nice. If something is blue and not green then it is kind. Query: Charlie is not blue.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: All green, nice people are cold. Fiona is white. If someone is cold and not smart then they are not kind. Bob is blue. Charlie is green. If Bob is cold then Bob is green. Kind people are not blue. If someone is not white then they are nice. All kind people are nice. Query: Fiona is not white.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: If Erin is quiet then Erin is blue. Erin is nice. Nice things are quiet. All red things are big. If something is red then it is quiet. If Erin is red and Erin is blue then Erin is nice. If something is blue and nice then it is cold. All big, red things are blue. If something is big and quiet then it is smart. Query: Erin is not blue.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: The rabbit visits the bald eagle. The mouse does not eat the rabbit. The mouse sees the rabbit. The rabbit visits the mouse. If the cow visits the bald eagle and the cow is kind then the bald eagle is kind. The mouse eats the cow. The rabbit eats the cow. The bald eagle sees the cow. The mouse is kind. The cow eats the bald eagle. The cow visits the bald eagle. The bald eagle visits the mouse. The rabbit does not see the cow. If something sees the mouse then the mouse does not eat the cow. The mouse is blue. The rabbit visits the cow. The cow does not see the bald eagle. Query: The cow visits the bald eagle.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: Young, smart people are cold. Gary is young. All kind, nice people are smart. All smart people are round. Erin is young. Round people are cold. Kind, nice people are rough. Round, cold people are young. Dave is kind. Bob is smart. Cold, young people are rough. Nice people are young. Query: Bob is not young.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: If something does not need the mouse then the mouse needs the bald eagle. If something sees the rabbit then the rabbit sees the mouse. The mouse eats the rabbit. If something needs the mouse and it sees the mouse then the mouse sees the rabbit. If the bald eagle needs the rabbit then the bald eagle is big. The bald eagle does not eat the rabbit. The rabbit needs the mouse. If something sees the bald eagle and it is not cold then the bald eagle is big. If the bald eagle does not need the mouse then the bald eagle sees the rabbit. Query: The mouse does not eat the rabbit.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: Nice, green people are not cold. If Gary is green then Gary is smart. Gary is green. Gary is not round. If someone is rough and smart then they are nice. Gary is rough. Query: Gary is not smart.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: Anne is round. All smart, blue people are round. All furry people are smart. Erin is smart. Round, big people are smart. Charlie is nice. Anne is big. If someone is smart then they are nice. Erin is big. Erin is kind. All nice people are round. Furry people are smart. Charlie is kind. All furry people are kind. Nice people are kind. Query: Anne is kind.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: Both green but also blue, Alan is nice and kind while being cold. Bob always goes out of his way to show that he is kind to people. I know Dave really well and you can say he is rough on the outside, cold on the inside and red all of the time. Fred is a young and nice man, and he is also very big shaped. A big person who is kind but blue is cold. If someone is cold and young, you'll see that they are kind. You will discover that green rough people are always red. Being green, blue, and red all point to how young someone was. When you know someone who's red, rough and nice, you'll also see that they're cold. Blue, kind, and cold people are habitually rough. Query: Bob is not rough.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: Erin is quiet. Bob is red. Erin is red. Charlie is young. All red people are smart. If someone is furry and not red then they are rough. Fiona is not furry. If Charlie is red then Charlie is young. Young, furry people are not cold. All furry people are not red. Charlie is rough. Fiona is quiet. If someone is smart and not cold then they are not young. If someone is not young then they are red. All rough people are quiet. Query: Erin is smart.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: Anne is blue. If someone is blue and big then they are red. Big people are red. Anne is nice. All big, rough people are blue. Anne is rough. Bob is big. If Anne is smart and Anne is red then Anne is rough. Gary is big. All white, smart people are big. Bob is blue. Dave is white. If someone is rough then they are smart. Dave is blue. If Dave is nice then Dave is blue. Gary is nice. If someone is nice and blue then they are rough. Anne is white. Query: Anne is red.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: Harry is furry. Bob is furry. If something is nice then it is red. Furry things are nice. Furry, red things are round. Query: Harry is nice.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: All blue, red things are young. If something is blue then it is red. The cat is blue. Query: The cat is young.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: If Dave is round then Dave is red. Harry is round. If Dave is round and Dave is quiet then Dave is blue. Harry is quiet. Dave is quiet. Query: Dave is not quiet.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: Charlie is big. Charlie is smart. Anne is rough. Dave is young. All big people are not cold. Charlie is young. Dave is kind. Harry is rough. Dave is not rough. Anne is smart. If someone is young and not big then they are not cold. Harry is kind. Query: Anne is not big.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: If something is round then it visits the lion. If the cow chases the lion then the lion is blue. The cow chases the lion. If something chases the cow then the cow chases the lion. If something chases the cow and it does not visit the lion then it does not visit the cow. The lion visits the cow. The cow sees the lion. If something is not green then it does not chase the cow. If something is not round then it sees the lion. If something visits the cow and it sees the cow then it is blue. If something is blue and it does not see the cow then the cow is round. Query: The cow is green.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: Dave is smart. Erin is cold. Charlie is red. If Erin is smart then Erin is red. Anne is not young. All quiet things are smart. If something is cold then it is quiet. Query: Erin is not quiet.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: Gary is nice. Harry is quiet. Kind things are quiet. If something is white then it is kind. Fiona is rough. Erin is nice. If Gary is nice then Gary is white. Query: Gary is quiet.",

"Given some premises, conduct reasoning to answer whether the given query is true, false or unknown. Premises: The cat does not like the rabbit. If someone visits the rabbit then they eat the rabbit. If someone does not eat the cat then the cat is rough. The rabbit visits the mouse. If the cat visits the rabbit then the cat does not eat the mouse. If someone likes the mouse then the mouse does not like the rabbit. If someone eats the cat and the cat does not eat the mouse then the cat visits the rabbit. If the mouse likes the cat then the cat is nice. If someone eats the rabbit then the rabbit likes the mouse. The mouse eats the cat. If the rabbit is kind and someone does not like the rabbit then they are green. Query: The cat likes the rabbit.",
]

In [7]:
output_texts2 = []
for prompt in tqdm(prompts_logic):
    input_text = 'Q: ' + prompt
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

    max_length = 600  # set the maximum length of the generated sequence

    output_ids = model.generate(
        input_ids,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,  # specify the end-of-sequence token ID
    )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    output_texts2.append(output_text)

  0%|          | 0/20 [00:00<?, ?it/s]

Ответ модели почти на все промпты: **I'm not sure how to start this, I am not sure if I am doing it correctly и т.п.**.

Слишком сложное задание для такой маленькой ллмки.

Для задач перевода поперебирал ллмки, но они практически всегда галлюцинируют.

Таким образом попробуем **тюнинг температуры** и **factual-nucleus sampling** на запросах о значениях слов. Способ из статьи **stitch in time says nine** я тоже попробовал - ллмка никак не меняет свой ответ (что уж говорить, раз гигачад подвержен этому, то и локальная маленькая ллмка на 3 миллиарда параметров тоже)

### Тюнинг температуры

Вместо `greedy strategy` будет сэмплировать по вероятностям.

100 - полный бред;

75 - все еще бред;

50 - все еще бред;

25 - все еще бред;

10 - все еще бред;

меньше 1 - генерирует одно и то же;

1-1.2 - неплохо. Немного увеличилась **diversity**, сохраняя **coherence**

In [24]:
output_texts = []
for prompt in tqdm(prompts_meanings):
    input_text = 'Q: ' + prompt
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

    max_length = 300  # set the maximum length of the generated sequence

    output_ids = model.generate(
        input_ids,
        max_length=max_length,
        temperature=1.2,
        do_sample = True,
        pad_token_id=tokenizer.eos_token_id,  # specify the end-of-sequence token ID
    )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    output_texts.append(output_text)

  0%|          | 0/20 [00:00<?, ?it/s]

### Factual-Nucleus sampling

https://arxiv.org/abs/2206.04624

In [25]:
end_of_sentence_idxes = tokenizer.encode('. ? !')[1:]
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=False, padding_side='left', device_map = 'auto')

Оптимальные гиперпараметры из статьи:

In [59]:
LAMBDA_DECAY = 0.9
P_RESET = 0.9
W_BOUND = 0.3

In [60]:
import torch
output_texts = []
for prompt in tqdm(prompts_meanings):
    input_text = 'Q: ' + prompt
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

    max_length = 300  # set the maximum length of the generated sequence
    cur_pos_in_sentence=1
    
    for _ in range(max_length):
        nucleus_p = max(W_BOUND, P_RESET * pow(LAMBDA_DECAY, cur_pos_in_sentence - 1))
        output_ids = model.generate(
            input_ids,
            max_length=len(input_ids[0]) + 1,
            temperature=1.0,
            do_sample = True,
            top_p = nucleus_p,
            pad_token_id=tokenizer.eos_token_id,
        )
        
        last_token = output_ids[:, -1]
        if last_token in end_of_sentence_idxes:
            cur_pos_in_sentence = 1
        else:
            cur_pos_in_sentence += 1
        
        
        input_ids = torch.cat([input_ids, last_token.unsqueeze(0)], dim=1)

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    output_texts.append(output_text)

  0%|          | 0/20 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

### Некоторые примеры фиксов

**Q: ology**

**llama (wrong)**

A: Ology is a word that is used to describe a particular type of blog. It is a portmanteau of "blog" and "ology".

**llama sampling with temperature (wrong)**

A: "The Wikipedia article is very good about explaining ology (link below):

*A ology (/ˈɑːlˌwɪdʒ/ or /ˈˌoʊlədʒ/) is a branch of study within a particular science or field of knowledge, or other type of science.

*In this article ology and scientific field are treated synonymously. A field of study is not a synonym of a branch of science; nor is a branch a synonym of an ology.
One can have a field of study in general, such as philosophy, as well as a branch of a scientific field, such as biochemistry.
In biology a field is a division of life science research: botany, zoology etc. Within the broader field of biologic science, there is specific subdiscipline of study"


**llama factual nucleus sampling (right)**:

Ology is a word that means “the study of.” It is a noun that is used to describe a field of study



**Q: bilon**

**llama (wrong)**

A: The term billon is used to describe a coin that was struck in a small quantity. The term is used to describe coins struck in the 1st century BC and 1st century AD

**llama sampling with temperature (right)**

A: For much of ancient history the coins were lost until much later, when they discovered that silver/base metal that was of good color and appearance and had a distinctive shape was of more economic value to the owner


**llama factual nucleus sampling (wrong)**:

It's a French word, meaning "of or relating to a bill"